In [1]:
pip install PyPDF2 langchain-community langchain-text-splitters langchain-google-genai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import openai

# For scraping
import requests
import pandas as pd
from bs4 import BeautifulSoup

# For data analysis and visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
import random
import PyPDF2
import pandas as pd
import re
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema import Document
from langchain.agents import initialize_agent, Tool, AgentType


In [3]:

# File path
file_path = "Merger guidelines - Final.pdf"

# Open the PDF file
with open(file_path, "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    text_data = []

    # Extract text from each page
    for page in reader.pages:
        raw_text = page.extract_text()
        if raw_text.strip():  # Ignore empty pages
            # Clean the text
            cleaned_text = re.sub(r'\n+', ' ', raw_text)  # Replace multiple newlines with a single space
            cleaned_text = cleaned_text.strip()  # Remove leading/trailing spaces
            text_data.append(cleaned_text)

# Create a DataFrame
df = pd.DataFrame(text_data, columns=["Content"])

# Split sections if applicable
df['Section'] = df['Content'].str.extract(r'(Article\s[IVX]+|Appendix|Exhibit\s[A-Z])', expand=False).fillna("General")

# Save the DataFrame to a CSV file
output_csv = "cleaned_pdf_to_dataframe.csv"
df.to_csv(output_csv, index=False)

# Display a sample of the DataFrame
print(df.head())


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Acer\AppData\Local\Temp\ipykernel_53020\4259480150.py:2: SyntaxWarning: invalid escape sequence '\s'
  file_path = "D:\super-duper-experimentation\dataset_gen\Merger guidelines - Final.pdf"
C:\Users\Acer\AppData\Local\Temp\ipykernel_53020\4259480150.py:2: SyntaxWarning: invalid escape sequence '\s'
  file_path = "D:\super-duper-experimentation\dataset_gen\Merger guidelines - Final.pdf"


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\super-duper-experimentation\\dataset_gen\\Merger guidelines - Final.pdf'

In [ ]:
CHUNK_SIZE = 4000
QUESTIONS_PER_CHUNK = 6
NUM_DOCUMENTS = 50

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=400,
    length_function=len,
    is_separator_regex=False,
)
documents = []
for index, page in df.iterrows():
    chunks = text_splitter.split_text(page["Content"])
    for chunk in chunks:
        documents.append(Document(page_content=chunk))
print(documents[0])

page_content='Merger Guidelines       November 2008 *        *November 2017 – amended to include Harper reforms to the   Competition and Consumer Act 2010'


In [ ]:
from dotenv import load_dotenv
load_dotenv('../../.env')

In [ ]:
# Initialize OpenAI or Gemini Chat model through LangChain


from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Define the template for generating questions
prompt_template = """
You are given a document. Please generate {questions_per_chunk} questions based on the above document.
The questions should be diverse and ask for different aspects of the document. Don't give vague references
to the document without description. Split each question with a newline.
Document: {document}
"""
prompt = PromptTemplate(input_variables=["document", "questions_per_chunk"], template=prompt_template)

# Create the LLMChain to combine the prompt with the model
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Process the documents and generate questions
queries = []


# Assuming `documents` is a list of Document objects
for doc in documents[:NUM_DOCUMENTS]:
    chunk = doc.page_content

    # Run the LLMChain to generate questions for this document chunk
    response = llm_chain.run({"document": chunk, "questions_per_chunk": QUESTIONS_PER_CHUNK})

    # Process the response
    question_list = [q.strip() for q in response.splitlines()]

    queries.append({"chunk": chunk, "questions": question_list})

# Print out the first query set
print(queries[0])


{'chunk': 'Merger Guidelines       November 2008 *        *November 2017 – amended to include Harper reforms to the   Competition and Consumer Act 2010', 'questions': ['What specific changes to the Competition and Consumer Act 2010 were incorporated into the Merger Guidelines in November 2017?', '', 'What is the significance of the November 2008 date in relation to the provided Merger Guidelines?', '', 'Are there any other versions of the Merger Guidelines besides the ones mentioned (November 2008 and the amended 2017 version)?', '', 'What is the primary purpose or function of the Merger Guidelines document?', '', "Who is responsible for issuing and maintaining these Merger Guidelines? (If the document doesn't state this explicitly, say so).", '', 'How might the Harper reforms, implemented in 2017, affect the assessment of mergers under the guidelines?']}


In [ ]:
print(queries[24])

{'chunk': 'Merger G uidelines 2008, updated 2017   21   products may belong to seve ral separate disaggregated markets and/or one  aggregated market consisting of a suite of goods and services generally supplied  together (sometimes referred to as a ‘cluster market’). To define the relevant markets  in these cases, the ACCC considers, among other things:   \uf0b7 the split between products  purchased or supplied separately, and products   purchased or supplied together   \uf0b7 the costs involved in purchasing  or supplying the product  separately   \uf0b7 any obstacles to purchasing  or supplying the product  separately   \uf0b7 any assets or specialisation required to supply each product.', 'questions': ['What criteria does the ACCC use to define relevant markets when products might belong to both separate and aggregated markets, according to the 2017 update of the 2008 Merger Guidelines?', '', "How does the proportion of products purchased or supplied separately versus together infl

In [ ]:
finalqueries = []

for query in queries:
  for question in query["questions"]:
    if question:
      if question[1] == ".":
        finalqueries.append({"chunk": query["chunk"], "question": question[2:]})
      else:
        finalqueries.append({"chunk": query["chunk"], "question": question})

In [ ]:
df = pd.DataFrame(finalqueries)

In [ ]:
df

,chunk,question
0,Merger Guidelines November 2008 * ...,What specific changes to the Competition and C...
1,Merger Guidelines November 2008 * ...,What is the significance of the November 2008 ...
2,Merger Guidelines November 2008 * ...,Are there any other versions of the Merger Gui...
3,Merger Guidelines November 2008 * ...,What is the primary purpose or function of the...
4,Merger Guidelines November 2008 * ...,Who is responsible for issuing and maintaining...
...,...,...
295,"Merger G uidelines 2008, updated 2017 45 ...","The document discusses ""countervailing power..."
296,"Merger G uidelines 2008, updated 2017 45 ...",What types of information might the ACCC req...
297,"Merger G uidelines 2008, updated 2017 45 ...",The document mentions that the ACCC's inform...
298,"Merger G uidelines 2008, updated 2017 45 ...",How does the document address the dynamic na...


In [ ]:
df.to_csv('final_questions_with_chunk.csv', index=False)